In [ ]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=85cdf3ca8ac44312e3004c89b7291775e1d7632660a004adfd6da7e301716cd4
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
import pandas as pd

In [ ]:
data = pd.read_excel('combined_comments3.xlsx')

In [ ]:
data = data.iloc[:1100,:]

In [ ]:
data = data.rename(columns={data.columns[0]: 'text', data.columns[1]: 'sentiment'})


In [ ]:
pip install deep-translator

In [ ]:
data.head()

,text,sentiment
0,Fuck La Tunisie Mouch blédi nikha b Fasma Sous...,abusive
1,ken hedha mouch rap enti fech tbi3 lzebi :p,abusive
2,fuck you chnawa teswa 7a9 plazma :p,abusive
3,chouf chanteur yghani l'amour lweld l9ahba fik...,abusive
4,3omrek matjib 7jar l3arka fiha kartouch ( chko...,normal


In [ ]:
import pandas as pd
import numpy as np
import json
import requests
from deep_translator import GoogleTranslator
from langdetect import detect
import re

In [ ]:
def LAT2AR(word):
  path = "https://api.yamli.com/transliterate.ashx?word={}&tool=api&account_id=000006&prot=https:&hostname=www.yamli.com&path=/&build=5515".format(word)
  response = requests.get(path)
  html = response.content
  try:
    newWord = json.loads(html.decode("utf-8"))["r"].split("|")[0].rsplit('/',1)[0]
  except:
    return word
  return (word if newWord == ""  else newWord)

In [ ]:
def translate(text):
  try :
    newtext = []
    text = re.sub(r"[,.;@/#?\!&$]+\ *", " ", text)
    text = text.split(' ')
    blist = [1 if (len(txt)>1 and detect(txt) in['fr','en']) else 0 for txt in text]
    try:
      for i in range(len(blist)):
        if blist[i] == 1:
          newtext.append(GoogleTranslator(source='auto', target='ar').translate(text[i]))
        else:
          newtext.append(LAT2AR(text[i].strip()))
    except:
      print(text[i])
      newtext.append(LAT2AR(text[i].strip()))

    return ' '.join(newtext)
  except:
    return " "

In [ ]:
translate("3aslema hedha test mta3 translation")

'يعطيه عصبة قيس سعيد'

In [ ]:
data['text_arabic'] = data['text'].apply(lambda x: translate(str(x)))

In [ ]:
data.to_csv("transalted.csv")